In [12]:
import pickle as pkl
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import TruncatedSVD, NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [13]:
with open('game_id_df.pkl', 'rb') as file:
    game_id_df = pkl.load(file)
with open('train_data_reviews.pkl', 'rb') as file2:
    train_data_reviews = pkl.load(file2)    

In [14]:
n_comp = 20
lsa = TruncatedSVD(n_components=n_comp)


In [15]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', 
                                   token_pattern="\\b[a-z][a-z]+\\b",
                                   lowercase=True,
                                   max_df = 0.6)
tfidf_data = tfidf_vectorizer.fit_transform(train_data_reviews)
lsa_tfidf_data = lsa.fit_transform(tfidf_data)

In [16]:
def get_game(names):
    #name = "The Lord of the Rings: The Battle for Middle-Earth"
    games=[]
    for name in names:
        datas= game_id_df[game_id_df.Title == name]
        if not datas.empty:
            data_index = game_id_df[game_id_df.Title == name].index[0]
            train_data_reviews[data_index]

            new_datapoint = [train_data_reviews[data_index]]
            new_vec = lsa.transform(tfidf_vectorizer.transform(new_datapoint))

            nn = NearestNeighbors(n_neighbors=5, metric='cosine', algorithm='brute')
            nn.fit(lsa_tfidf_data)

            result = nn.kneighbors(new_vec)

            
            for r in result[1][0]:
                game = game_id_df.Title[r]
                if game not in games:
                    games.append(game)
    print(games)

In [21]:
gamess = ['Titanfall 2', 'Battlefield: Bad Company','Battlefield 3']
get_game(gamess)

['Titanfall 2', 'Battlefield: Bad Company', 'Titanfall', 'Battlefield 3', 'Battlefield: Bad Company 2', 'Battlefield 4']


In [30]:
import requests as req
import json 

with open('jsongames.json') as json_file:
    data = json.load(json_file)
games_list = []
for games in data['games']:
    games_list.append(games)
games_list
get_game(games_list)

['Age of Empires III', 'Hearts of Iron II', 'Sins of a Solar Empire', 'Empire: Total War', 'Age of Mythology', 'Alien: Isolation', 'Alan Wake', 'Metro Redux', "Clive Barker's Undying", "Peter Jackson's King Kong: The Official Game of the Movie", 'Diablo III', 'Hex: Shards of Fate', 'Marvel Heroes 2015', 'The Elder Scrolls: Legends', 'Heroes of the Storm', 'Dragon Age: Origins', "Baldur's Gate II: Shadows of Amn", 'Dragon Age II', 'Endless Space 2', "Age of Wonders II: The Wizard's Throne", 'Galactic Civilizations II: Dread Lords', 'Galactic Civilizations', "King's Bounty: Armored Princess", 'Grand Theft Auto III', 'Grand Theft Auto: San Andreas', 'Grand Theft Auto IV', 'Grand Theft Auto: Vice City', 'Life is Strange', 'The Walking Dead: A Telltale Games Series', 'Life is Strange: Episode 5 - Polarized', 'Minecraft', 'Fortnite', 'Minecraft: PlayStation 4 Edition', 'Starbound', 'Minecraft: PlayStation 3 Edition']


In [28]:
data

{'games': ['ARK: Survival Evolved',
  'Age of Empires II: HD Edition',
  'Age of Empires III',
  'Age of Wonders III',
  'Alien: Isolation',
  'Aliens vs Predator',
  'Diablo III',
  'Dragon Age: Origins',
  'Endless Space',
  'Endless Space 2',
  'Grand Theft Auto III',
  'Heroes of Might and Magic V',
  'Life is Strange',
  'Minecraft']}